Separate accidents that have happened on holidays from the dataset and save them into a new dataset with a marking to differentiate different holidays.

In [ ]:
import pandas as pd

def getHolidayName(date):
    holidays = {
        "01.01": "New Year's Day",
        "24.02": "Independence day",
        "01.05": "Spring Holiday",
        "23.06": "Victory Day",
        "24.06": "Midsummer day",
        "20.08": "Restoration of Independence Day",
        "24.12": "Christmas Eve",
        "25.12": "Christmas",
        "26.12": "Christmas 2"
    }

    return holidays.get(date)

filename = "Liiklusõnnetused_2011_2021_korrastatud.csv"

dates = ["01.01", "24.02", "01.05", "23.06", "24.06", "20.08", "24.12", "25.12"]

df = pd.read_csv(filename, sep=";")

column = "date"

date_df = []

for i in range(len(dates)):
    rows = df[df[column] == dates[i]]
    df = df[df[column] != dates[i]]
    date_df.append(rows)


In [4]:
import pandas as pd
filename = "data_preparation/cleaned_data_file.csv"
df = pd.read_csv(filename, sep=",")

In [19]:
from datetime import date, timedelta

def movingHolidays():
    movingholidays = {"Good Friday": [], "Easter": [], "Pentecost": []}

    for y in range(2011, 2022):
        #Following code is from https://www.assa.org.au/edm#Calculator
        first_dig = y // 100  # first 2 digits of year
        remain_19 = y % 19  # remainder of year / 19

        # calculate PFM date
        temp = (first_dig - 15) // 2 + 202 - 11 * remain_19

        if first_dig in [21, 24, 25, 27, 28, 29, 30, 31, 32, 34, 35, 38]:
            temp -= 1
        elif first_dig in [33, 36, 37, 39, 40]:
            temp -= 2
        temp %= 30

        t_a = temp + 21
        if temp == 29:
            t_a -= 1
        if temp == 28 and remain_19 > 10:
            t_a -= 1

        # find the next Sunday
        t_b = (t_a - 19) % 7

        t_c = (40 - first_dig) % 4
        if t_c == 3:
            t_c += 1
        if t_c > 1:
            t_c += 1

        temp = y % 100
        t_d = (temp + temp // 4) % 7

        t_e = ((20 - t_b - t_c - t_d) % 7) + 1
        d = t_a + t_e

        if d > 31:
            d -= 31
            m = 4
        else:
            m = 3

        easter = date(y, m, d)
        goodfriday = easter - timedelta(2)
        pentecost = easter + timedelta(49)

        movingholidays["Good Friday"].append(goodfriday.strftime("%d.%m.%y"))
        movingholidays["Pentecost"].append(pentecost.strftime("%d.%m.%y"))
        movingholidays["Easter"].append(easter.strftime("%d.%m.%y"))

    return movingholidays

print(movingHolidays())

{'Good Friday': ['22.04.11', '06.04.12', '29.03.13', '18.04.14', '03.04.15', '25.03.16', '14.04.17', '30.03.18', '19.04.19', '10.04.20', '02.04.21'], 'Easter': ['24.04.11', '08.04.12', '31.03.13', '20.04.14', '05.04.15', '27.03.16', '16.04.17', '01.04.18', '21.04.19', '12.04.20', '04.04.21'], 'Pentecost': ['12.06.11', '27.05.12', '19.05.13', '08.06.14', '24.05.15', '15.05.16', '04.06.17', '20.05.18', '09.06.19', '31.05.20', '23.05.21']}
